In [1]:
import numpy as np
import os
import serial
import math
import matplotlib.pyplot as plt
import time
import cv2

In [2]:
capture = cv2.VideoCapture(1)

In [5]:
# 定义字符串的转换方式
# 先定义最大速度
max_speed=10
def convert_to_two_digits(number):
    if 10 > number >= 0:
        return '+0' + str(number)
    if 100 > number >= 10:
        return '+' + str(min(number,max_speed))
    if 0 > number > -10:
        return '-0' + str(number)[1]
    if -10 >= number > -100:
        return '-'+str(min(abs(number),max_speed))
# 在整个图片中获取红色点
# 在整个image所获取的图片中[:,:,1]是绿色图片,[:,:,2]是红色图片
template_image = cv2.imread('point.png')
template_gray = cv2.cvtColor(template_image, cv2.COLOR_BGR2GRAY)
threshold = 0.8
def get_red_point(capture, red_center):
    _,image=capture.read()
    # # 找到红色的点
    # image_red_true=image[:,:,2].astype(np.float32)
    # image_back=(image[:,:,1].astype(np.float32)+image[:,:,0].astype(np.float32))/2
    # image_back=image_back.astype(np.float32)
    # image_red=np.abs(image_red_true-image_back)
    # # image_red=image[:,:,1]
    # image_red=image_red.astype(np.uint8)
    # red = np.array(np.argwhere(image_red>70))
    # 红色中心点坐标
    image1_red=image[:,:,2].astype(np.float32)
    image1_green=image[:,:,1].astype(np.float32)
    image1_blue=image[:,:,0].astype(np.float32)
    image2=image1_red-(image1_blue+image1_green)/2
    image2[image2<0]=0
    image2=image2*2
    back_gray=image2.astype(np.uint8)
    try:
        result = cv2.matchTemplate(back_gray, template_gray, cv2.TM_CCOEFF_NORMED)
        locations = np.where(result >= threshold)
        locations = list(zip(*locations[::-1]))  # 转换为(x, y)坐标
        red_center=np.array([int(locations[0][1] + template_image.shape[0] / 2),int(locations[0][0] + template_image.shape[1] / 2)])
        # red_center=np.array([int(locations[0][0] + template_image.shape[1] / 2),int(locations[0][1] + template_image.shape[0] / 2)])
    except cv2.error:
        red_center=red_center
    # if red.shape[0]>2:
    #     red_center=np.ones(2)
    #     red_center[0]=int(np.sum(red[:,0])/(np.size(red[:,0])+0.001))
    #     red_center[1]=int(np.sum(red[:,1])/(np.size(red[:,0])+0.001))
    # else:
    #     pass
    return red_center
# 先降低摄像头曝光度，然后把图片rgb通道分离，用r通道减掉b通道，然后把得到的图片中像素小于等于0的像素值直接置为0（可以用max函数与0做比较），接着把把图片每个像素点的值乘以一个系数（我当时取的3,不合适就自己调一下），然后再一次二值化（二值化阈值自己调），这个时候图像上应该会有个小白点，这个白点应该就是激光点。通道相减有个好处，相减的过程中白色和黑色都会被消去,相当于做了差分,把小于零的点全部置为0后再乘以系数相当于一个放大作用，只放大r值大于g值的部分,这个部分就是激光点,然后做一次二值化得到的白色区域应该就是激光点,因为激光点中心比较亮，你们在摄像头里看到的可能是中间是白色，外围是红色，用这个方法处理因为白色会被消除，所以最后结果可能是个白圈,就以白圈中心做为激光点位置就行,或者就做个膨胀,把圈填满
# 在整个image所获取的图片中[:,:,1]是绿色图片,[:,:,2]是红色图片

In [6]:
# 文件夹路径
# folder_path = 'C:\Parallel_World\BaiduSyncdisk\My_World\Software_World\Python\Computer_Vision\Photo_Data\All_photo\\'
# # # 获取文件夹中的所有.npy文件
# # file_names = [f for f in os.listdir(folder_path) if f.endswith('.npy')]
# circle=np.load(folder_path+'circle.npy')
# circle_center=np.load(folder_path+'circle_center.npy')
# edges=np.load(folder_path+'edges.npy')
# edges_show=np.load(folder_path+'edges_show.npy')
# gray=np.load(folder_path+'gray.npy')
# gray1=np.load(folder_path+'gray1.npy')
# image=np.load(folder_path+'image.npy')
# image1=np.load(folder_path+'image1.npy')
# image_back=np.load(folder_path+'image_back.npy')
# image_red=np.load(folder_path+'image_red.npy')
# image_red_true=np.load(folder_path+'image_red_true.npy')
# kernel=np.load(folder_path+'kernel.npy')
# red=np.load(folder_path+'red.npy')
# red_center=np.load(folder_path+'red_center.npy')
# red_matrix=np.load(folder_path+'red_matrix.npy')

In [7]:
# 定义卷积核
kernel = np.array([
[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0],
[0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
[0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0],
[0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0],
[0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0],
[0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0],
[0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0],
[0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0],
[0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0],
[0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0],
[0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0],
[0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0],
])*0.01
a=11
b=11

In [11]:
# 红 绿 蓝
# '''
# _,image1=capture.read()
# 对于所获得的图像，首先计算他的几何中心
# 先转为灰度图
# gray1=cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
# # 获取边界
# edges=cv2.Canny(gray1, 240, 200)
# # 打开开关
red_center=np.ones(2)
red_center[0]=240
red_center[1]=320
while True:# 首先我们读取摄像头的图片
    _,image=capture.read()
    # 对于所获得的图像，首先计算他的几何中心
    # 先转为灰度图
    gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges_show=gray
    # # 获取边界
    # image_red_true=image[:,:,1].astype(np.float32)
    # image_back=(image[:,:,2].astype(np.float32)+image[:,:,0].astype(np.float32))/2
    # image_back=image_back.astype(np.float32)
    # image_red=np.abs(image_red_true-image_back)
    # # image_red=image[:,:,1]
    # image_red=image_red.astype(numpy.uint8)
    # red = np.array(np.argwhere(image_red>70))
    # circle= np.array(np.where(edges))
    # 获取红色中心点的坐标
    # if red.shape[0]>2:
    #     red_center=np.ones(2)
    #     red_center[0]=int(np.sum(red[:,0])/(np.size(red[:,0])+0.001))
    #     red_center[1]=int(np.sum(red[:,1])/(np.size(red[:,0])+0.001))
    # else:
    #     red_center=np.ones(2)
    #     red_center[0]=240
    #     red_center[1]=320
    # red_center,back_gray=get_red_point(capture,red_center)
    #red_center=np.array([red_center1[0,1],red_center1[0,0]])

    # 获得边框中心点的坐标
    # circle_center=np.ones(2)
    # circle_center[0]=int(np.sum(circle[:,0])/(np.size(circle[:,0])+0.001))
    # circle_center[1]=int(np.sum(circle[:,1])/(np.size(circle[:,1])+0.001))
    # 之后，我们利用卷积操作，计算我们的中心点的边缘与边框
    # 首先，创建一个同等大小的矩阵
    _,image=capture.read()
    image1_red=image[:,:,2].astype(np.float32)
    image1_green=image[:,:,1].astype(np.float32)
    image1_blue=image[:,:,0].astype(np.float32)
    image2=image1_green-(image1_blue+image1_red)/2
    image2[image2<0]=0
    image2=image2*2

    red_matrix=np.zeros((480,640))
    # 叠加点的图像
    if (int(red_center[0]-a)>0)&(int(red_center[0]+b)<480)&(int(red_center[1]-a)>0)&(int(red_center[1]+b)<640):
        red_matrix[int(red_center[0]-a):int(red_center[0]+b),int(red_center[1]-a):int(red_center[1]+b)]=kernel
        edges_show[int(red_center[0]-a):int(red_center[0]+b),int(red_center[1]-a):int(red_center[1]+b)]=kernel*25400
    else:
        red_matrix[240-a:240+b,320-a:320+b]=kernel
        edges_show[240-a:240+b,320-a:320+b]=kernel*25400
    # 定义得分
    # score=np.sum(edges*red_matrix)
    # 打印得分
    # '''
    # gray[0,0]=score
    # cv2.imshow('image2', image2)
    cv2.imshow('image', image)
    cv2.imshow('edges_show', edges_show)
    # cv2.imshow('back_gray', back_gray.astype(np.uint8))
    cv2.imshow('image2', image2.astype(np.uint8))
    cv2.imshow('red_matrix', red_matrix*20000)
    cv2.waitKey(1)
    # '''
#'''

KeyboardInterrupt: 